In [ ]:
# else子句不能能在if语句中使用，还能在for、while和try语句中使用。 
# 在for和while的判定条件不成立时，进入else子句。在try没有捕获到异常时进入else子句。

# 上下文管理器 - 协议包含__enter__和__exit__两个方法
# with语句会设置一个临时的上下文，交给上下文管理器对象控制，并负责清理上下文。with语句开始运行时，会调用上下文管理器的
# __enter__方法（返回上下文管理器，或者其他对象）；运行结束后，会调用__exit__方法（在上下文管理器上调用，
# 而不是在__enter__方法返回的对象上调用）。

#
with open('email.py') as fp:
    src = fp.read(60)

print(len(src))
print(fp)   # fp 仍存在，但不能进行IO
print(fp.closed, fp.encoding)
print(fp.read(10)) # 不能进行IO


In [ ]:
# 与函数和模块不同，with块没有定义新的作用域！
class LookingGlass:

    def __enter__(self):  # with开始进入调用
        import sys
        self.original_write = sys.stdout.write  # 保存原方法待用
        sys.stdout.write = self.reverse_write  # 打猴子补丁，运行期修改输出方法的实现
        return 'JABBERWOCKY'  # 返回对象，作为as子句绑定用值

    def reverse_write(self, text):  # 自定义方法 反转输出
        self.original_write(text[::-1])

    # with结束调用，除self外三个参数在正常情况下都是None；如果with内抛出了异常，这三个参数是异常数据
    def __exit__(self, exc_type, exc_value, traceback):  
        import sys  # <7>
        sys.stdout.write = self.original_write  # 恢复原方法
        if exc_type is ZeroDivisionError:  # 如果有该异常，打印一个信息
            print('Please DO NOT divide by zero!')
            return True  # 通知解释器该异常已处理
        # 如果__exit__方法返回None, True之外的值，with块中的任何异常都会向上冒泡

with LookingGlass() as what:    # __enter__ 返回结果绑定到what上
    print('Alice, Kitty and Snowdrop') # 在with中输出方法被改为反转输出
    print(what)
    print('------------') # __exit__ 调用后，输出复原
print(what)
print('Back to normal')

# 上下文管理器的一些应用：
# - sqlite3模块中用于管理事务
# - threading模块中用于维护锁、条件和信号
# - 为Decimal对象的算数运算设置环境
# - 为测试临时给对象打补丁


# contextlib模块实用工具 - 用于创建上下文管理器等

# @contextmanager装饰器能减少创建上下文管理器的样板代码量（不用编写一个类，实现__enter__、__exit__方法），只需实现有一个yield语句的
# 生成器，生成想让__enter__方法返回的值。在yield语句前面的所有代码在with块开始时执行（类似__enter__方法），后面的代码在with块结束时
# 执行（类似__exit__方法）。
print('-----gen1--------')
import contextlib

@contextlib.contextmanager 
def looking_glass():
    import sys
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    yield 'JABBERWOCKY'  # 返回给as子句用于绑定的值
    sys.stdout.write = original_write  # <6>

with looking_glass() as what:    # __enter__ 返回结果绑定到what上
    print('Alice, Kitty and Snowdrop') # 在with中输出方法被改为反转输出
    print(what)
    print('------------') # __exit__ 调用后，输出复原
print(what)
print('Back to normal')

# 上一个版本的函数未处理with中的异常，Python解释器捕获后，会在yield表达式再次抛出，函数会被终止导致yield之后的内容
# 没有机会执行，输出方法不能恢复到原来。失去了上下文管理的作用。
print('-----gen2--------')
@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    msg = ''  # <1>
    try:  # 捕获异常进行处理
        yield 'JABBERWOCKY'
    except ZeroDivisionError:  # <2>
        msg = 'Please DO NOT divide by zero!'
    finally: # 无论如何，恢复输出方法
        sys.stdout.write = original_write  # <3>
        if msg:
            print(msg)  # <4>

# 和前面__exit__方法不同，装饰@contextmanager默认所有异常都得到处理了。如果不想压制异常，则需要用throw方法把异常发给生成器。
